In [1]:
import itertools, collections
import numpy as np

In [2]:
class NutritionVector(collections.namedtuple('NutritionVector', 'fat protein carbonhydrate calories')):
    """ Nutrition facts information """
    def scale(self, n):
        """ Apply scaling factor or matrix transformation """
        return np.dot(n, self)
    
    def __str__(self):
        return 'fat {:.2f}g\tprotein {:.2f}g\tcarbs {:.2f}g\tcalories {:.2f}kcal'.format(*self)

class NutritousObject(object):
    """ Objects that holds a NutritionVector """
    def __init__(self, name, nutrition, scale):
        super(NutritousObject, self).__init__()
        self.name = name
        self.nutrition = NutritionVector._make(nutrition)
        self._scale = scale
        
    @property
    def value(self):
        """ Scaled nutrition facts """
        return self.nutrition.scale(self._scale)
    
    def __str__(self):
        return '{}: {}'.format(self.name, self.nutrition) if self.name else str(self.nutrition)

class NutritousArray(list):
    @staticmethod
    def of(ctor, inits):
        """ Generic NutritousArray factory for NutritousObject """
        return NutritousArray(itertools.starmap(ctor, inits))
        
    def sum(self):
        """ Take summation of nutrition facts for all NutritousObject in the array """
        return np.sum(np.vstack([o.value for o in self]), axis = 0)
    
    def __getitem__(self, indices):
        """ Fancy indexing using cached hash-join operator """
        if not hasattr(self, 'nutritous_index'):
            self.nutritous_index = {o.name:o for o in self}
        return NutritousArray(map(self.nutritous_index.get, indices))

In [3]:
class Food(NutritousObject):
    def __init__(self, name, nutrition, scale):
        super(Food, self).__init__(name, nutrition, scale)
    
all_food = NutritousArray.of(Food, [
    ("chicken thighs / breast", (10, 19, 0, 160), 4.049928571),
    ("pasta", (1, 7, 42, 200), 2.026785714),
    ("eggs", (5, 6, 0.6, 78), 2),
    ("tomato", (0.2, 1.1, 4.8, 22), 1),
    ("broccoli", (0.5, 4.2, 10, 50), 1),
    ("cheese", (9, 7, 0.4, 113), 1),
    ("raspberry", (0.8, 1.5, 15, 65), 0.2),
    ("banana", (0.4, 1.3, 27, 105), 1),
    ("shrimp", (0.3, 24, 0.2, 99), 0.7),
    ("tilapia", (2.3, 23, 0, 112), 4),
    ("rice", (0, 3, 33, 150), 4),
])

for food in all_food:
    print(food)

chicken thighs / breast: fat 10.00g	protein 19.00g	carbs 0.00g	calories 160.00kcal
pasta: fat 1.00g	protein 7.00g	carbs 42.00g	calories 200.00kcal
eggs: fat 5.00g	protein 6.00g	carbs 0.60g	calories 78.00kcal
tomato: fat 0.20g	protein 1.10g	carbs 4.80g	calories 22.00kcal
broccoli: fat 0.50g	protein 4.20g	carbs 10.00g	calories 50.00kcal
cheese: fat 9.00g	protein 7.00g	carbs 0.40g	calories 113.00kcal
raspberry: fat 0.80g	protein 1.50g	carbs 15.00g	calories 65.00kcal
banana: fat 0.40g	protein 1.30g	carbs 27.00g	calories 105.00kcal
shrimp: fat 0.30g	protein 24.00g	carbs 0.20g	calories 99.00kcal
tilapia: fat 2.30g	protein 23.00g	carbs 0.00g	calories 112.00kcal
rice: fat 0.00g	protein 3.00g	carbs 33.00g	calories 150.00kcal


In [4]:
class Meal(NutritousObject):
    def __init__(self, name, all_food, transform = 1.0):
        self.all_food = all_food
        super(Meal, self).__init__(name, self.all_food.sum(), transform)
        
all_meals = NutritousArray.of(Meal, [
    ("Lunch 1", all_food[
        ["chicken thighs / breast",
        "rice",
        "shrimp",
        "broccoli",
        "cheese",]]
    ),
    ("Dinner 1", all_food[
        ["tilapia",
        "pasta",
        "eggs",
        "tomato",
        "banana",]]
    ),
    ("Dinner 2", all_food[
        ["chicken thighs / breast",
        "pasta",
        "broccoli",
        "banana",]]
    ),
])

for meal in all_meals:
    print(meal)

Lunch 1: fat 50.21g	protein 116.95g	carbs 142.54g	calories 1480.29kcal
Dinner 1: fat 21.83g	protein 120.59g	carbs 118.12g	calories 1136.36kcal
Dinner 2: fat 43.43g	protein 96.64g	carbs 122.12g	calories 1208.35kcal


In [5]:
class Plan(NutritousObject):
    def __init__(self, meals):
        self.meals = meals
        super(Plan, self).__init__('', meals.sum(), 1.0)

plans = NutritousArray.of(Plan, [
    (all_meals["Lunch 1", "Dinner 1"], ),
    (all_meals["Lunch 1", "Dinner 2"], ),
])

for plan in plans:
    print(plan)

fat 72.04g	protein 237.54g	carbs 260.66g	calories 2616.65kcal
fat 93.64g	protein 213.58g	carbs 264.66g	calories 2688.63kcal


In [6]:
goal = NutritionVector(calories = 2405, protein = 172,  fat = 63, carbonhydrate = 287).scale(1.0)

for plan in plans:
    print('Difference from goal:', NutritionVector._make(plan.value - goal))

Difference from goal: fat 9.04g	protein 65.54g	carbs -26.34g	calories 211.65kcal
Difference from goal: fat 30.64g	protein 41.58g	carbs -22.34g	calories 283.63kcal
